In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json

## Grab some books from THR

In [2]:
def getBook(id):
    resp = requests.get('http://tarheelreader.org/book-as-json?id=%d' % id)
    return resp.json()

def findBooks(page):
    resp = requests.get('http://tarheelreader.org/find/?json=1&page=%d' % page)
    return [book['ID'] for book in resp.json()['books']]

books = []
for i in range(1, 2):
    books.extend([ getBook(i) for i in findBooks(i) ])

## Write them to the new instance

In [8]:
passwd = '2d46 RG2n wzZ2 c4re'
userid = 'admin'

# 2016-08-04 15:59:38

from datetime import datetime
from pprint import pprint

def createBook(o):
    b = {}
    b['title'] = o['title']
    b['pictures'] = [ page['url'] for page in o['pages'] ]
    b['text'] = [ page['text'] for page in o['pages'] ]
    b['pseudonym'] = o['author']
    b['language'] = o['language']
    b['date'] = datetime.strptime(o['created'], "%Y-%m-%d %H:%M:%S").isoformat()
    b['modified'] = datetime.strptime(o['modified'], "%Y-%m-%d %H:%M:%S").isoformat()
    b['categories'] = [3]
    b['rating_count'] = o['rating_count']
    b['rating_sum'] = o['rating_total']
    b['reviewed'] = o['reviewed']
    b['status'] = 'publish'
    b['audience'] = o['audience']
    return b
    
for book in books:
    b = createBook(book)
    resp = requests.post('http://localhost:8000/wp-json/wp/v2/posts', json=b, 
                         auth=HTTPBasicAuth(userid, passwd))
    

In [5]:
b = createBook(books[0])
pprint(b)

{'categories': [3],
 'comment_status': 'closed',
 'date': '2016-08-03T15:35:06',
 'language': 'en',
 'modified': '2016-08-04T15:59:38',
 'pictures': ['/cache/images/09/9103287209_67f31d4f71_t.jpg',
              '/cache/images/09/9103287209_67f31d4f71.jpg',
              '/cache/images/24/43311724_28f21b3ba0.jpg',
              '/cache/images/60/6240747460_75e5cc597c.jpg',
              '/cache/images/47/104635547_3660490d7f.jpg',
              '/cache/images/82/7512299482_9e2282919a.jpg',
              '/cache/images/43/9158471643_06485bc8cc.jpg',
              '/cache/images/47/5744039447_be22f59fa7.jpg',
              '/cache/images/57/5543405357_fdef694614.jpg',
              '/cache/images/76/3178812576_6026543dfd.jpg',
              '/cache/images/43/6240460843_d16beb35fb.jpg',
              '/cache/images/25/3528060125_e66f63a1a4.jpg'],
 'ping_status': 'closed',
 'pseudonym': 'Mary Hobbs',
 'rating_count': 30,
 'rating_sum': 66,
 'reviewed': True,
 'status': 'publish',
 'text': 